<a href="https://colab.research.google.com/github/HieuHongHao/Amazon-review-instruments/blob/main/Amazon_Instrument_Review_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
from math import nan
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation,Dense,Embedding,LSTM
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.python.training.tracking.base import no_manual_dependency_tracking_scope
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences




    

In [ ]:
pwd

'/content'

In [ ]:
#Extract reviews and overall scores
drive.mount('/content/gdrive')
data = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks')
data = data.dropna() # drop missing values 
data = data[['reviewText','overall']]
reviews = np.array(data['reviewText'])
scores = data['overall']
scores = np.array(scores)
scores = np.array([1 if score >= 3 else 0 for score in scores])


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


ParserError: ignored

In [ ]:
pwd

'/content'

In [ ]:
vocabSize = 10000
reviews = [one_hot(doc,vocabSize,filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n') for doc in reviews]

In [ ]:
maxSentLen = 0
for doc in reviews:
  if(len(doc) > maxSentLen):
    maxSentLen = len(doc)
embedded_docs = pad_sequences(reviews,padding="pre",maxlen = maxSentLen)


In [ ]:
embedded_docs

array([[   0,    0,    0, ..., 3235, 2942, 3146],
       [   0,    0,    0, ...,  752, 9556, 3440],
       [   0,    0,    0, ..., 5841, 2683, 9310],
       ...,
       [   0,    0,    0, ..., 2683, 1913,  794],
       [   0,    0,    0, ..., 1526, 1496, 1851],
       [   0,    0,    0, ..., 8766, 8766, 5825]], dtype=int32)

In [ ]:
Vector_features = 100
model = Sequential()
model.add(Embedding(vocabSize,Vector_features,input_length = maxSentLen))
model.add(LSTM(100))  
model.add(Dense(1,activation="sigmoid")) 
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 2059, 100)         1000000   
                                                                 
 lstm (LSTM)                 (None, 100)               80400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 1,080,501
Trainable params: 1,080,501
Non-trainable params: 0
_________________________________________________________________


In [ ]:
x_train,x_test,y_train,y_test = train_test_split(np.array(embedded_docs),scores,test_size = 0.33,random_state = 42)
x_train = np.array(x_train)
x_test = np.array(x_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

In [ ]:
model.fit(x_train,y_train,validation_data=(x_test,y_test),epochs= 10, batch_size= 64)

Epoch 1/10
108/108 [==============================] - 333s 3s/step - loss: 0.2297 - accuracy: 0.9494 - val_loss: 0.1740 - val_accuracy: 0.9573
Epoch 2/10
 36/108 [=========>....................] - ETA: 3:10 - loss: 0.1751 - accuracy: 0.9549

KeyboardInterrupt: ignored

In [ ]:
y_pred = model.predict(x_test)
y_pred = (model.predict(x_test) > 0.5).astype("int32")


In [ ]:
from tensorflow.keras.metrics import Accuracy
measures = Accuracy()
measures.update_state(y_test,y_pred)
measures.result().numpy()


0.93244445

In [ ]:
%cd /content/gdrive/My Drive

/content/gdrive/My Drive


In [ ]:
% cd Colab\ Notebooks


/content/gdrive/My Drive/Colab Notebooks


In [ ]:
ls


 Amazon_Instrument_Review_Classification.ipynb
'Bản sao của Core Learning Algorithms.ipynb'
'Bản sao của Course 3 - Week 4 - Lesson 2 - Notebook.ipynb'
'Bản sao của Spotify Project Part 1 (1).ipynb'
'Bản sao của Spotify Project Part 1.ipynb'
'Bản sao của Spotify Project Part 2.ipynb'
'Bản sao của Spotify Project Part 3.ipynb'
'Copy of Spotify Project Part 3.ipynb'
 SiameseNN.ipynb
 tensorFlowIntro.ipynb
 Untitled0.ipynb
 Untitled1.ipynb
 Untitled2.ipynb


In [ ]:
!git init 


Initialized empty Git repository in /content/gdrive/MyDrive/Colab Notebooks/.git/


In [ ]:
!git status 

On branch master

No commits yet

Untracked files:
  (use "git add <file>..." to include in what will be committed)

	Amazon_Instrument_Review_Classification.ipynb
	"Ba\314\211n sao cu\314\211a Core Learning Algorithms.ipynb"
	"Ba\314\211n sao cu\314\211a Course 3 - Week 4 - Lesson 2 - Notebook.ipynb"
	"Ba\314\211n sao cu\314\211a Spotify Project Part 1 (1).ipynb"
	"Ba\314\211n sao cu\314\211a Spotify Project Part 1.ipynb"
	"Ba\314\211n sao cu\314\211a Spotify Project Part 2.ipynb"
	"Ba\314\211n sao cu\314\211a Spotify Project Part 3.ipynb"
	Copy of Spotify Project Part 3.ipynb
	SiameseNN.ipynb
	Untitled0.ipynb
	Untitled1.ipynb
	Untitled2.ipynb
	tensorFlowIntro.ipynb

nothing added to commit but untracked files present (use "git add" to track)


In [ ]:
!git add Amazon_Instrument_Review_Classification.ipynb

In [ ]:
!git status

On branch master

No commits yet

Changes to be committed:
  (use "git rm --cached <file>..." to unstage)

	new file:   Amazon_Instrument_Review_Classification.ipynb

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git checkout -- <file>..." to discard changes in working directory)

	modified:   Amazon_Instrument_Review_Classification.ipynb

Untracked files:
  (use "git add <file>..." to include in what will be committed)

	"Ba\314\211n sao cu\314\211a Core Learning Algorithms.ipynb"
	"Ba\314\211n sao cu\314\211a Course 3 - Week 4 - Lesson 2 - Notebook.ipynb"
	"Ba\314\211n sao cu\314\211a Spotify Project Part 1 (1).ipynb"
	"Ba\314\211n sao cu\314\211a Spotify Project Part 1.ipynb"
	"Ba\314\211n sao cu\314\211a Spotify Project Part 2.ipynb"
	"Ba\314\211n sao cu\314\211a Spotify Project Part 3.ipynb"
	Copy of Spotify Project Part 3.ipynb
	SiameseNN.ipynb
	Untitled0.ipynb
	Untitled1.ipynb
	Untitled2.ipynb
	tensorFlowIntro.ipynb



In [ ]:
!git status


On branch master

No commits yet

Changes to be committed:
  (use "git rm --cached <file>..." to unstage)

	new file:   Amazon_Instrument_Review_Classification.ipynb

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git checkout -- <file>..." to discard changes in working directory)

	modified:   Amazon_Instrument_Review_Classification.ipynb

Untracked files:
  (use "git add <file>..." to include in what will be committed)

	"Ba\314\211n sao cu\314\211a Core Learning Algorithms.ipynb"
	"Ba\314\211n sao cu\314\211a Course 3 - Week 4 - Lesson 2 - Notebook.ipynb"
	"Ba\314\211n sao cu\314\211a Spotify Project Part 1 (1).ipynb"
	"Ba\314\211n sao cu\314\211a Spotify Project Part 1.ipynb"
	"Ba\314\211n sao cu\314\211a Spotify Project Part 2.ipynb"
	"Ba\314\211n sao cu\314\211a Spotify Project Part 3.ipynb"
	Copy of Spotify Project Part 3.ipynb
	Musical_instruments_reviews.csv
	SiameseNN.ipynb
	Untitled0.ipynb
	Untitled1.ipynb
	Untitled2.ipynb

In [ ]:
!git add Musical_instruments_reviews.csv

In [ ]:
!git remote add origin https://github.com/HieuHongHao/Amazon-review-instruments.git

In [ ]:
!git config --global user.email "hietnguyen@umass.edu"
!git config --global user.name "HieuHongHao"

In [ ]:
!git commit -m "First commit"

[master (root-commit) 02f5772] First commit
 2 files changed, 10263 insertions(+)
 create mode 100644 Amazon_Instrument_Review_Classification.ipynb
 create mode 100644 Musical_instruments_reviews.csv


In [ ]:
!git config --list

user.email=hietnguyen@umass.edu
user.name=HieuHongHao
core.repositoryformatversion=0
core.filemode=true
core.bare=false
core.logallrefupdates=true
remote.origin.url=https://github.com/HieuHongHao/Amazon-review-instruments.git
remote.origin.fetch=+refs/heads/*:refs/remotes/origin/*


In [ ]:
!git remote rm origin 

In [ ]:
!git remote add origin https://HieuHongHao:27092002Ut@@github.com/HieuHongHao/Amazon-review-instruments.git

In [ ]:
!git push origin master 

fatal: unable to access 'https://HieuHongHao:27092002Ut@@github.com/HieuHongHao/Amazon-review-instruments.git/': Could not resolve host: @github.com
